# 准备工作

In [65]:
import pandas as pd

In [66]:
file = "~/Data/iris.data"   #这里要改为你自己电脑上存储的文件路径
data = pd.read_csv(file,header=None)
data.columns = ["sepal_length","sepal_width","petal_length","petal_width","class"]

In [67]:
data[:3]

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa


In [68]:
data.describe()

,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


# 数据预处理

#### 首先，把class这个类别型的变量转为0，1，2这样的数值型

查看class共用多少枚举值，然后进行数值化处理

In [69]:
data["class"].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [70]:
data["label"] = data["class"].map({"Iris-setosa":0,"Iris-versicolor":1,"Iris-virginica":2})
data[:3]

,sepal_length,sepal_width,petal_length,petal_width,class,label
0,5.1,3.5,1.4,0.2,Iris-setosa,0
1,4.9,3.0,1.4,0.2,Iris-setosa,0
2,4.7,3.2,1.3,0.2,Iris-setosa,0


拆分特征变量、标签变量

In [71]:
X = data[["sepal_length","sepal_width","petal_length","petal_width"]]
y = data["label"]

# 动起手来

还记得吗？当我们用留出法将数据集拆分为训练集、测试集时，使用的是sklearn中train_test_split接口，那么现在要变成K-折交叉验证，就要换换操作方法了。
sklearn中提供了一个KFold的接口可以供我们来按指定拆分的份数，官方文档详见：https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html#sklearn.model_selection.KFold

In [72]:
from sklearn.model_selection import KFold
# 在每次拆分前，先对数据进行随机排序，即进行了shuffle操作
# 但每一次拆分动作执行后，拆分的结果就应该固定下来，所以要给一个随机数种子，通过random_state来设置
kf = KFold(n_splits=10, shuffle = True, random_state = 0)

事实上，留一法就是在K折拆分中，把每一条记录当作一折，所以 n_splits = len(data)。

《机器学习》西瓜书第26页上说，每一次拆分K折数据，这个划分动作本身也可能引入误差，因此可以进行p次随机k折交叉验证（搞科学的人，就是特别在意抽样的随机性，以及随之可能存在的统计偏差）。
那么我们也这样来操作吧。但是，如果是留一法呢？就只有一种划分方法了呀，那就没有必要再进行p次实验了。

##### 先来p次k折交叉验证

所以我们一共会进行 p * k 次训练，right?

In [73]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
p = 10
#定义一个list来保存每一次训练后的准确率
score_result = []

for i in range(0,p):
    kf = KFold(n_splits=10, shuffle = True, random_state = 0)
    #kf.split 将数据进行k折拆分，返回类型是一个迭代器，每一次迭代中都包括了训练样本、测试样本所在的行索引号，根据这些索引号就可以取到对应的数据
    for train_index, test_index in kf.split(X_scaled):
        X_train, X_test = X_scaled[train_index],X_scaled[test_index]
        y_train, y_test = y[train_index], y[test_index]
        #数据标准化处理
        scaler = preprocessing.StandardScaler().fit(X_train)
        X_train_scaled = scaler.transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        #训练logistic回归算法
        clf = LogisticRegression(penalty='l2',solver = "lbfgs",multi_class="auto")
        model = clf.fit(X_train_scaled,y_train)
        y_predict = model.predict(X_test_scaled)
        score = accuracy_score(y_test,y_predict)
        score_result.append(score)

print("共获得了%d个准确率分数" % len(score_result))
print(score_result[:10])

共获得了100个准确率分数
[1.0, 1.0, 0.9333333333333333, 0.8, 1.0, 1.0, 1.0, 1.0, 0.9333333333333333, 0.9333333333333333]


求平均准确率

In [74]:
import numpy as np
print("平均准确率：%f" % (np.mean(score_result)))

平均准确率：0.960000


Not bad, right?

#### 再来试试留一法

In [75]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

#定义一个list来保存每一次训练后的准确率
score_result = []
kf = KFold(n_splits=len(data), shuffle = True, random_state = 0)
#kf.split 将数据进行k折拆分，返回类型是一个迭代器，每一次迭代中都包括了训练样本、测试样本所在的行索引号，根据这些索引号就可以取到对应的数据
for train_index, test_index in kf.split(X_scaled):
    X_train, X_test = X_scaled[train_index],X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]
    #训练logistic回归算法
    clf = LogisticRegression(penalty='l2',solver = "lbfgs",multi_class="auto")
    model = clf.fit(X_train,y_train)
    y_predict = model.predict(X_test)
    score = accuracy_score(y_test,y_predict)
    score_result.append(score)

print("共获得了%d个准确率分数" % len(score_result))
print(score_result[:10])
import numpy as np
print("平均准确率：%f" % (np.mean(score_result)))

共获得了150个准确率分数
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
平均准确率：0.953333
